# Description

This script was created to consume Salesforce API, query the necessary object, and insert the data into SQL Server to be used in reports in Tableau.

The reason for this is that using Salesforce in Tableau sometimes is challenging, since Salesforce does not allow complex joins between the objects, making some analysis harder.

The script queries Salesforce, getting all the data. Then it queries also the SQL Server table created to store the data and compares it with Salesforce to check if any line was deleted, modified or if it's a new line.

If the line was deleted, the script deletes from SQL Server. If it was modified, it updates the necessary columns. If it's a new line, it inserts the line into SQL Server.

From this script, it was generated an EXE file to run at specific times during the day, to guarantee SQL Server is up to date with Salesforce.

# Importing Data from Salesforce and Inserting into SQL Server

## Importing the Libraries

In [3]:
from simple_salesforce import Salesforce
import pandas as pd
import re
import pyodbc
from datetime import datetime
import numpy as np
import warnings
import time
import json
import os
import sys

## Importing the credentials to access Salesforce

Since the code uses credentials that experies from time to time, it makes a connection with an external file to get the credentials to query Salesforce.

In case the credentials change, we just need to change the external file and there's no need to replace the whole EXE.

In [4]:
# Function to get the path to the credentials file
def get_credentials_path():
    return os.path.join(os.getcwd(), 'MyCredentials.json')

# Function to read the credentials
def read_credentials():
    credentials_path = get_credentials_path()
    with open(credentials_path, 'r') as file:
        credentials = json.load(file)
    return credentials

In [5]:
# Read the credentials
credentials = read_credentials()
username_sf = credentials['username']
password_sf = credentials['password']
security_token_sf = credentials['security_token']

## Making the connection

In [6]:
sf = Salesforce(username=f'{username_sf}', password=f'{password_sf}', security_token=f'{security_token_sf}')

## Querying the tables

### Creating the variables to use in the code

In [7]:
# Define the connection parameters

server_sql = 'sample_server'
database_sql = 'Sample_Salesforce'
username_sql = 'SampleUserName'
password_sql = 'SamplePassword'

# Filter out the pyodbc warning
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

# Establish the connection with Windows authentication
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_sql};DATABASE={database_sql};UID={username_sql};PWD={password_sql};'
)

# Create a cursor object to interact with the database
cursor = conn.cursor()


#----------------------- CALL TABLE -----------------------#

# getting the time the code started

calls_start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Selecting the last created date in the Calls table when the job ran for the last time
with conn.cursor() as cursor:
    created_call_date_query = """
    SELECT MAX([LastLineCreated_Date])
    FROM [Sample_Salesforce].[dbo].[SF_InsertJobs_Log]
    WHERE [Table_Name] = 'F2FCalls'
    """
    cursor.execute(created_call_date_query)
    last_created_call_date = cursor.fetchone()[0]
    last_created_call_date_extra = last_created_call_date
    last_created_call_date = last_created_call_date.strftime('%Y-%m-%dT%H:%M:%SZ')

# Selecting the last modified date in the Calls table when the job ran for the last time
with conn.cursor() as cursor:
    modified_call_date_query = """
    SELECT MAX([LastLineModified_Date])
    FROM [Sample_Salesforce].[dbo].[SF_InsertJobs_Log]
    WHERE [Table_Name] = 'F2FCalls'
    """
    cursor.execute(modified_call_date_query)
    last_modified_call_date = cursor.fetchone()[0]
    last_modified_call_date_extra = last_modified_call_date
    last_modified_call_date = last_modified_call_date.strftime('%Y-%m-%dT%H:%M:%SZ')


# Select the existing IDs in Calls table in SQL
select_IDs_calls = """
SELECT [ID]

FROM [Sample_Salesforce].[dbo].[SF_F2FCalls]
"""
df_calls_ids = pd.read_sql_query(select_IDs_calls, conn)
call_IDs = df_calls_ids['ID']


# Close the cursor and connection
cursor.close()
conn.close()

### F2F Calls

#### New Lines

In [13]:
calls=sf.query_all(f"""
SELECT 

Id, Account__c, Assigned_SalesRep_Id_Site__c, OwnerId, Site__c, Contact__c, Name, Call_Street__c, Call_City__c, Call_Post_Code__c, Call_Date__c, 
Call_Details_HTML_stripped__c, Call_Type__c, Call_Objectives__c, Off_Road_Days__c, Off_Road_Reason__c, Status__c, Contact_Name__c, toLabel(Account__r.Customer_Type__c), CreatedById, 
CreatedDate, LastModifiedById, LastModifiedDate


FROM Call__c

WHERE CreatedDate >= {last_created_call_date}

ORDER BY Id

""")

In [14]:
# Transforming the query into a data frame
df_calls_created = pd.DataFrame(calls['records'])

# Dropping the element 'attibutes'
df_calls_created = df_calls_created.drop('attributes', axis=1)

# Excluding any HTML tags from the Call Details column (some lines come with tags like <p>example</p>)
df_calls_created['Call_Details_HTML_stripped__c'] = df_calls_created['Call_Details_HTML_stripped__c'].apply(lambda x: re.sub('<[^<]+?>', '', x) if isinstance(x, str) else None)

# Rename the column to 'Customer_Type__c'
df_calls_created.rename(columns={'Account__r': 'Customer_Type__c'}, inplace=True)


# Replace the contents of Customer_Type__c for the values inside the dictionary, using None for missing keys
# This is done because we query using a relationship with Account, so the API brings a dictionary structure istead of the value itself 

df_calls_created['Customer_Type__c'] = [
    row.get('Customer_Type__c', None) if isinstance(row, dict) else None
    for row in df_calls_created['Customer_Type__c']
]

# converting date columns to the SQL Server pattern
df_calls_created['CreatedDate'] = pd.to_datetime(df_calls_created['CreatedDate'])
df_calls_created['CreatedDate'] = df_calls_created['CreatedDate'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_calls_created['LastModifiedDate'] = pd.to_datetime(df_calls_created['LastModifiedDate'])
df_calls_created['LastModifiedDate'] = df_calls_created['LastModifiedDate'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_calls_created['Call_Date__c'] = pd.to_datetime(df_calls_created['Call_Date__c'])
df_calls_created['Call_Date__c'] = df_calls_created['Call_Date__c'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Filling any NaN with None
df_calls_created = df_calls_created.replace(pd.NA, None)

# filtering the IDs already in SQL
df_calls_created = df_calls_created[~df_calls_created['Id'].isin(call_IDs)]

df_calls_created

,Id,Account__c,Assigned_SalesRep_Id_Site__c,OwnerId,Site__c,Contact__c,Name,Call_Street__c,Call_City__c,Call_Post_Code__c,...,Call_Objectives__c,Off_Road_Days__c,Off_Road_Reason__c,Status__c,Contact_Name__c,Customer_Type__c,CreatedById,CreatedDate,LastModifiedById,LastModifiedDate


In [ ]:
print('The new lines were queried. The dataframe is ready!')

#### Updates

##### Querying Salesforce for modified fields

In [10]:
calls=sf.query_all(f"""
SELECT 

Id, Account__c, Assigned_SalesRep_Id_Site__c, OwnerId, Site__c, Contact__c, Name, Call_Street__c, Call_City__c, Call_Post_Code__c, Call_Date__c, 
Call_Details_HTML_stripped__c, Call_Type__c, Call_Objectives__c, Off_Road_Days__c, Off_Road_Reason__c, Status__c, Contact_Name__c, toLabel(Account__r.Customer_Type__c), CreatedById, 
CreatedDate, LastModifiedById, LastModifiedDate


FROM Call__c

WHERE LastModifiedDate >= {last_modified_call_date}

ORDER BY Id

""")

In [11]:
# Transforming the query into a data frame
df_calls_updated = pd.DataFrame(calls['records'])

# Dropping the element 'attibutes'
df_calls_updated = df_calls_updated.drop('attributes', axis=1)

# Excluding any HTML tags from the Call Details column (some lines come with tags like <p>example</p>)
df_calls_updated['Call_Details_HTML_stripped__c'] = df_calls_updated['Call_Details_HTML_stripped__c'].apply(lambda x: re.sub('<[^<]+?>', '', x) if isinstance(x, str) else None)


# Rename the column 'Account__r' to 'Customer_Type__c'
df_calls_updated.rename(columns={'Account__r': 'Customer_Type__c'}, inplace=True)


# Replace the contents of Customer_Type__c for the values inside the dictionary, using None for missing keys
# This is done because we query using a relationship with Account, so the API brings a dictionary structure istead of the value itself 

df_calls_updated['Customer_Type__c'] = [
    row.get('Customer_Type__c', None) if isinstance(row, dict) else None
    for row in df_calls_updated['Customer_Type__c']
]


# converting date columns to the SQL Server pattern
df_calls_updated['CreatedDate'] = pd.to_datetime(df_calls_updated['CreatedDate'])
df_calls_updated['CreatedDate'] = df_calls_updated['CreatedDate'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_calls_updated['LastModifiedDate'] = pd.to_datetime(df_calls_updated['LastModifiedDate'])
df_calls_updated['LastModifiedDate'] = df_calls_updated['LastModifiedDate'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_calls_updated['Call_Date__c'] = pd.to_datetime(df_calls_updated['Call_Date__c'])
df_calls_updated['Call_Date__c'] = df_calls_updated['Call_Date__c'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Filling any NaN with None
df_calls_updated = df_calls_updated.replace(pd.NA, None)

# filtering the IDs that are new
df_calls_updated = df_calls_updated[~df_calls_updated['Id'].isin(df_calls_created['Id'])]

# sorting and reseting the index
df_calls_updated = df_calls_updated.sort_values(by='Id')
df_calls_updated = df_calls_updated.reset_index(drop=True)

df_calls_updated

,Id,Account__c,Assigned_SalesRep_Id_Site__c,OwnerId,Site__c,Contact__c,Name,Call_Street__c,Call_City__c,Call_Post_Code__c,...,Call_Objectives__c,Off_Road_Days__c,Off_Road_Reason__c,Status__c,Contact_Name__c,Customer_Type__c,CreatedById,CreatedDate,LastModifiedById,LastModifiedDate
0,a1qW2000001ZksEIAS,0015i000013jpaoAAA,0055i00000C3niP,0055i00000C3niPAAR,a1n5i000002N6uZAAS,003W2000009jEbCIAU,Dairy Farmers Corner - Civil & Civic Infrastru...,Dairy Farmers Corner,NEWCASTLE WEST,2302,...,None,None,None,Completed,Andrew Nolan,Civil Contractor,0055i00000C3niPAAR,2025-03-18 01:44:15,0055i00000C3niPAAR,2025-03-18 01:44:15
1,a1qW2000001Zl6jIAC,0015i000013js3JAAQ,0055i00000CPpla,0055i00000C3nhnAAB,a1n5i000002N0MBAA0,003W200000AJ2ZpIAL,Prospect Reservor NSW - Abergeldie Contractors...,Prospect Hwy,PROSPECT,2148,...,None,None,None,Completed,Jeff Holman,Civil Contractor,0055i00000C3nhnAAB,2025-03-18 00:27:42,0055i00000C3nhnAAB,2025-03-18 00:27:43
2,a1qW2000001ZlTJIA0,0015i000013jmkkAAA,None,0055i00000C3niTAAR,None,003W2000008gagnIAA,City Of Port Philip (Dead Custoemr),333 Bay Street,PORT MELBOURNE,3207,...,None,None,None,Completed,Dead Custoemr,Council,0055i00000C3niTAAR,2025-03-18 00:33:58,0055i00000C3niTAAR,2025-03-18 00:33:58
3,a1qW2000001ZlYAIA0,0015i000013jlRPAAY,None,0055i00000C3niHAAR,None,0035i0000BVqpPKAQZ,D & A Annand Plumbers & Gasfitters (D & A Anna...,8 Raymond Elliot Crt,PARK ORCHARDS,3114,...,None,None,None,Completed,D & A Annad Plumbers,Plumbers,0055i00000C3niHAAR,2025-03-18 00:42:14,0055i00000C3niHAAR,2025-03-18 00:42:14
4,a1qW2000001ZlgDIAS,0015i000013meIbAAI,0055i00000C3ni2,0055i00000C3ni2AAB,a1nW2000002nugFIAQ,003W2000008yUbKIAU,90 Refinery rd Corio Fulton Hogan - Fulton Hog...,90 Refinery Road,Corio,3214,...,None,None,None,Completed,Syed Khumaini,Civil Contractor,0055i00000C3ni2AAB,2025-03-18 00:40:11,0055i00000C3ni2AAB,2025-03-18 00:40:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,a1qW2000001a153IAA,0015i000013joV2AAI,None,0055i00000BwjWeAAJ,None,003J3000002RZYGIA4,JMG Maintenance & fabrication (Michael Deakin),1 Strathmore road,MUSWELLBROOK,2333,...,None,None,None,Completed,Michael Deakin,Miscellaneous,0055i00000BwjWeAAJ,2025-03-18 04:29:35,0055i00000BwjWeAAJ,2025-03-18 04:29:35
113,a1qW2000001a1UrIAI,0015i000013jqpdAAA,None,0055i00000BwjWeAAJ,None,0035i0000BVohYDAQZ,Muswellbrook Shire Council (Matthew Grady),PO Box 122,MUSWELLBROOK,2333,...,None,None,None,Completed,Matthew Grady,Council,0055i00000BwjWeAAJ,2025-03-18 04:39:16,0055i00000BwjWeAAJ,2025-03-18 04:39:17
114,a1qW2000001a1WTIAY,0015i00001DclDOAAZ,None,0055i00000BwjWeAAJ,None,003J3000002SC9TIAW,Glencore - Ravensworth Open Cut ( ROC ) (Matt ...,Lemington Road,Ravensworth,2330,...,None,None,None,Completed,Matt Baskerville,Mining/Site services,0055i00000BwjWeAAJ,2025-03-18 04:40:22,0055i00000BwjWeAAJ,2025-03-18 04:40:23
115,a1qW2000001a1oDIAQ,0015i000013jpsIAAQ,0055i00000BwjWe,0055i00000BwjWeAAJ,a1nW2000004PF4AIAW,003W200000AJnKEIA1,KCE - HVO - Rehabilitation and water resivoir ...,Lemington Road,Lemington,2330,...,None,None,None,Completed,Ray Coyle,Civil Contractor,0055i00000BwjWeAAJ,2025-03-18 04:45:36,0055i00000BwjWeAAJ,2025-03-18 04:45:38


##### Querying SQL to compare with Salesforce

In [12]:
callLines_column_names = {
    'Id': 'ID',
    'Account__c': 'Account_ID',
    'Assigned_SalesRep_Id_Site__c': 'Site_SalesRep_ID',
    'OwnerId': 'Call_Owner_ID',
    'Site__c': 'Site_ID',
    'Contact__c': 'Contact_ID',
    'Name': 'Call_Name',
    'Call_Street__c': 'Call_Street',
    'Call_City__c': 'Call_Suburb',
    'Call_Post_Code__c': 'Call_Postcode',
    'Call_Date__c': 'Call_Date',
    'Call_Details_HTML_stripped__c': 'Call_Details',
    'Call_Type__c': 'Call_Type',
    'Call_Objectives__c': 'Call_Objectives',
    'Off_Road_Days__c': 'Off_Road_Days',
    'Off_Road_Reason__c': 'Off_Road_Reason',
    'Status__c': 'Call_Status',
    'Contact_Name__c': 'Contact_Name',
    'Customer_Type__c': 'Customer_Type',
    'CreatedById': 'CreatedBy_ID',
    'CreatedDate': 'Created_Date',
    'LastModifiedById': 'Last_ModifiedBy_ID',
    'LastModifiedDate': 'Last_Modified_Date'
}

In [15]:
if df_calls_updated.empty:
    df_up_callLines_SQL = df_calls_updated.copy()
    df_up_callLines_SQL = df_up_callLines_SQL.rename(columns = callLines_column_names)

else:

    # Establish the connection with Windows authentication
    conn = pyodbc.connect(
        f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_sql};DATABASE={database_sql};UID={username_sql};PWD={password_sql};'
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    updated_call_lines = f"""
    SELECT [ID]
          ,[Account_ID]
          ,[Site_SalesRep_ID]
          ,[Call_Owner_ID]
          ,[Site_ID]
          ,[Contact_ID]
          ,[Call_Name]
          ,[Call_Street]
          ,[Call_Suburb]
          ,[Call_Postcode]
          ,[Call_Date]
          ,[Call_Details]
          ,[Call_Type]
          ,[Call_Objectives]
          ,[Off_Road_Days]
          ,[Off_Road_Reason]
          ,[Call_Status]
          ,[Contact_Name]
          ,[Customer_Type]
          ,[CreatedBy_ID]
          ,[Created_Date]
          ,[Last_ModifiedBy_ID]
          ,[Last_Modified_Date]
    FROM [Sample_Salesforce].[dbo].[SF_F2FCalls]
    WHERE [ID] IN {"('"+tuple(df_calls_updated.Id)[0]+"')" if len(tuple(df_calls_updated.Id)) == 1 else tuple(df_calls_updated.Id)}
    ORDER BY [ID]
    """

    df_up_callLines_SQL = pd.read_sql_query(updated_call_lines, conn)
    df_up_callLines_SQL = df_up_callLines_SQL.sort_values(by='ID')
    df_up_callLines_SQL = df_up_callLines_SQL.reset_index(drop=True)

    # Close the cursor and connection
    cursor.close()
    conn.close()

df_up_callLines_SQL

,ID,Account_ID,Site_SalesRep_ID,Call_Owner_ID,Site_ID,Contact_ID,Call_Name,Call_Street,Call_Suburb,Call_Postcode,...,Call_Objectives,Off_Road_Days,Off_Road_Reason,Call_Status,Contact_Name,Customer_Type,CreatedBy_ID,Created_Date,Last_ModifiedBy_ID,Last_Modified_Date
0,a1qW2000001ZksEIAS,0015i000013jpaoAAA,0055i00000C3niP,0055i00000C3niPAAR,a1n5i000002N6uZAAS,003W2000009jEbCIAU,Dairy Farmers Corner - Civil & Civic Infrastru...,Dairy Farmers Corner,NEWCASTLE WEST,2302,...,None,None,None,Completed,Andrew Nolan,Civil Contractor,0055i00000C3niPAAR,2025-03-18 01:44:15,0055i00000C3niPAAR,2025-03-18 01:44:15
1,a1qW2000001Zl6jIAC,0015i000013js3JAAQ,0055i00000CPpla,0055i00000C3nhnAAB,a1n5i000002N0MBAA0,003W200000AJ2ZpIAL,Prospect Reservor NSW - Abergeldie Contractors...,Prospect Hwy,PROSPECT,2148,...,None,None,None,Completed,Jeff Holman,Civil Contractor,0055i00000C3nhnAAB,2025-03-18 00:27:42,0055i00000C3nhnAAB,2025-03-18 00:27:43
2,a1qW2000001ZlTJIA0,0015i000013jmkkAAA,None,0055i00000C3niTAAR,None,003W2000008gagnIAA,City Of Port Philip (Dead Custoemr),333 Bay Street,PORT MELBOURNE,3207,...,None,None,None,Completed,Dead Custoemr,Council,0055i00000C3niTAAR,2025-03-18 00:33:58,0055i00000C3niTAAR,2025-03-18 00:33:58
3,a1qW2000001ZlYAIA0,0015i000013jlRPAAY,None,0055i00000C3niHAAR,None,0035i0000BVqpPKAQZ,D & A Annand Plumbers & Gasfitters (D & A Anna...,8 Raymond Elliot Crt,PARK ORCHARDS,3114,...,None,None,None,Completed,D & A Annad Plumbers,Plumbers,0055i00000C3niHAAR,2025-03-18 00:42:14,0055i00000C3niHAAR,2025-03-18 00:42:14
4,a1qW2000001ZlgDIAS,0015i000013meIbAAI,0055i00000C3ni2,0055i00000C3ni2AAB,a1nW2000002nugFIAQ,003W2000008yUbKIAU,90 Refinery rd Corio Fulton Hogan - Fulton Hog...,90 Refinery Road,Corio,3214,...,None,None,None,Completed,Syed Khumaini,Civil Contractor,0055i00000C3ni2AAB,2025-03-18 00:40:11,0055i00000C3ni2AAB,2025-03-18 00:40:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,a1qW2000001a153IAA,0015i000013joV2AAI,None,0055i00000BwjWeAAJ,None,003J3000002RZYGIA4,JMG Maintenance & fabrication (Michael Deakin),1 Strathmore road,MUSWELLBROOK,2333,...,None,None,None,Completed,Michael Deakin,Miscellaneous,0055i00000BwjWeAAJ,2025-03-18 04:29:35,0055i00000BwjWeAAJ,2025-03-18 04:29:35
113,a1qW2000001a1UrIAI,0015i000013jqpdAAA,None,0055i00000BwjWeAAJ,None,0035i0000BVohYDAQZ,Muswellbrook Shire Council (Matthew Grady),PO Box 122,MUSWELLBROOK,2333,...,None,None,None,Completed,Matthew Grady,Council,0055i00000BwjWeAAJ,2025-03-18 04:39:16,0055i00000BwjWeAAJ,2025-03-18 04:39:17
114,a1qW2000001a1WTIAY,0015i00001DclDOAAZ,None,0055i00000BwjWeAAJ,None,003J3000002SC9TIAW,Glencore - Ravensworth Open Cut ( ROC ) (Matt ...,Lemington Road,Ravensworth,2330,...,None,None,None,Completed,Matt Baskerville,Mining/Site services,0055i00000BwjWeAAJ,2025-03-18 04:40:22,0055i00000BwjWeAAJ,2025-03-18 04:40:23
115,a1qW2000001a1oDIAQ,0015i000013jpsIAAQ,0055i00000BwjWe,0055i00000BwjWeAAJ,a1nW2000004PF4AIAW,003W200000AJnKEIA1,KCE - HVO - Rehabilitation and water resivoir ...,Lemington Road,Lemington,2330,...,None,None,None,Completed,Ray Coyle,Civil Contractor,0055i00000BwjWeAAJ,2025-03-18 04:45:36,0055i00000BwjWeAAJ,2025-03-18 04:45:38


##### Dropping equal lines

In [16]:
df_call_compare = df_calls_updated.rename(columns=callLines_column_names).reset_index(drop=True)

# Get boolean mask where all columns are equal
mask_calls = (df_up_callLines_SQL.fillna('NULL') == df_call_compare.fillna('NULL')).all(axis=1)

# Remove lines where mask is True
df_calls_updated = df_calls_updated[~mask_calls]

df_calls_updated

,Id,Account__c,Assigned_SalesRep_Id_Site__c,OwnerId,Site__c,Contact__c,Name,Call_Street__c,Call_City__c,Call_Post_Code__c,...,Call_Objectives__c,Off_Road_Days__c,Off_Road_Reason__c,Status__c,Contact_Name__c,Customer_Type__c,CreatedById,CreatedDate,LastModifiedById,LastModifiedDate
1,a1qW2000001Zl6jIAC,0015i000013js3JAAQ,0055i00000CPpla,0055i00000C3nhnAAB,a1n5i000002N0MBAA0,003W200000AJ2ZpIAL,Prospect Reservor NSW - Abergeldie Contractors...,Prospect Hwy,PROSPECT,2148,...,None,None,None,Completed,Jeff Holman,Civil Contractor,0055i00000C3nhnAAB,2025-03-18 00:27:42,0055i00000C3nhnAAB,2025-03-18 00:27:43


### Looking for deleted lines in Salesforce

If a line was deleted from Salesforce, it has to be deleted from SQL Server as well.

It will query the lines from Salesrforce and compare to what we have in SQL Server

#### Querying the IDs from Salesforce

In [17]:
calls_SF_id=sf.query_all(f"""
SELECT 

Id

FROM Call__c

ORDER BY Id

""")


# Transforming the query into a data frame
calls_SF_id = pd.DataFrame(calls_SF_id['records'])

# Dropping the element 'attibutes'
calls_SF_id = calls_SF_id.drop('attributes', axis=1)

calls_SF_id = set(calls_SF_id['Id'])

#### Querying the IDs from SQL Server

In [18]:
# Establish the connection with Windows authentication
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_sql};DATABASE={database_sql};UID={username_sql};PWD={password_sql};'
)

# Create a cursor object to interact with the database
cursor = conn.cursor()

updated_call_lines = f"""
SELECT [ID]
FROM [Sample_Salesforce].[dbo].[SF_F2FCalls]
ORDER BY [ID]
"""

call_SQL_id = pd.read_sql_query(updated_call_lines, conn)
call_SQL_id = call_SQL_id.sort_values(by='ID')
call_SQL_id = call_SQL_id.reset_index(drop=True)

call_SQL_id = set(call_SQL_id['ID'])

# Close the cursor and connection
cursor.close()
conn.close()

### Comparing the variables to see if there is a difference

In [19]:
# Find IDs in SQL that are not in Salesforce
ids_to_delete = call_SQL_id - calls_SF_id

### Deleting calls from SQL Server if they don't exist in Salesforce

In [20]:
if ids_to_delete:
    # Connect to SQL database
    conn = pyodbc.connect(
        f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_sql};DATABASE={database_sql};UID={username_sql};PWD={password_sql};'
    )
    cursor = conn.cursor()

    # Convert the set to a string format that can be used in the SQL query
    ids_to_delete_str = ','.join([f"'{id}'" for id in ids_to_delete])

    # Form the SQL DELETE query
    delete_query = f"DELETE FROM [Sample_Salesforce].[dbo].[SF_F2FCalls] WHERE [ID] IN ({ids_to_delete_str})"

    # Execute the query
    cursor.execute(delete_query)
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

In [21]:
print('The updated lines were queried. The dataframe is ready!')

The updated lines were queried. The dataframe is ready!


## Inserting and updating SQL

### F2F Calls

#### Inserting new lines

In [22]:
if df_calls_created.empty:
    pass
    
else:

    # Establish the connection with Windows authentication
    conn = pyodbc.connect(
        f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_sql};DATABASE={database_sql};UID={username_sql};PWD={password_sql};'
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    call_inserts = ''

    for index, row in df_calls_created.iterrows():

        ID = f"'{row['Id']}'" if row['Id'] is not None else 'NULL'

        Account_ID = f"'{row['Account__c']}'" if row['Account__c'] is not None else 'NULL'

        Site_SalesRep_ID = f"'{row['Assigned_SalesRep_Id_Site__c']}'" if row['Assigned_SalesRep_Id_Site__c'] is not None else 'NULL'

        Call_Owner_ID = f"'{row['OwnerId']}'" if row['OwnerId'] is not None else 'NULL'

        Site_ID = f"'{row['Site__c']}'" if row['Site__c'] is not None else 'NULL'

        Contact_ID = f"'{row['Contact__c']}'" if row['Contact__c'] is not None else 'NULL'

        Call_Name = str(f"{row['Name']}").replace("'", "''")
        Call_Name = f"'{Call_Name}'"  if row['Name'] is not None else 'NULL'

        Call_Street = str(f"{row['Call_Street__c']}").replace("'", "''")
        Call_Street = f"'{Call_Street}'" if row['Call_Street__c'] is not None else 'NULL'

        Call_Suburb = str(f"{row['Call_City__c']}").replace("'", "''")
        Call_Suburb = f"'{Call_Suburb}'" if row['Call_City__c'] is not None else 'NULL'

        Call_Postcode = f"'{row['Call_Post_Code__c']}'" if row['Call_Post_Code__c'] is not None else 'NULL'

        Call_Date = f"'{row['Call_Date__c']}'" if row['Call_Date__c'] is not None else 'NULL'

        Call_Details = str(f"{row['Call_Details_HTML_stripped__c']}").replace("'", "''")
        Call_Details = f"'{Call_Details}'" if row['Call_Details_HTML_stripped__c'] is not None else 'NULL'

        Call_Type = f"'{row['Call_Type__c']}'" if row['Call_Type__c'] is not None else 'NULL'

        Call_Objectives = str(f"{row['Call_Objectives__c']}").replace("'", "''")
        Call_Objectives = f"'{Call_Objectives}'" if row['Call_Objectives__c'] is not None else 'NULL'

        Off_Road_Days = f"'{row['Off_Road_Days__c']}'" if row['Off_Road_Days__c'] is not None else 'NULL'

        Off_Road_Reason = str(f"{row['Off_Road_Reason__c']}").replace("'", "''")
        Off_Road_Reason = f"'{Off_Road_Reason}'" if row['Off_Road_Reason__c'] is not None else 'NULL'

        Call_Status = f"'{row['Status__c']}'" if row['Status__c'] is not None else 'NULL'

        Contact_Name = str(f"{row['Contact_Name__c']}").replace("'", "''")
        Contact_Name = f"'{Contact_Name}'" if row['Contact_Name__c'] is not None else 'NULL'

        Customer_Type = f"'{row['Customer_Type__c']}'" if row['Customer_Type__c'] is not None else 'NULL'

        CreatedBy_ID = f"'{row['CreatedById']}'" if row['CreatedById'] is not None else 'NULL'

        Created_Date = f"'{row['CreatedDate']}'" if row['CreatedDate'] is not None else 'NULL'

        Last_ModifiedBy_ID = f"'{row['LastModifiedById']}'" if row['LastModifiedById'] is not None else 'NULL'

        Last_Modified_Date = f"'{row['LastModifiedDate']}'" if row['LastModifiedDate'] is not None else 'NULL'



        insert = f"""INSERT INTO [Sample_Salesforce].[dbo].[SF_F2FCalls]
        ([ID]
        ,[Account_ID]
        ,[Site_SalesRep_ID]
        ,[Call_Owner_ID]
        ,[Site_ID]
        ,[Contact_ID]
        ,[Call_Name]
        ,[Call_Street]
        ,[Call_Suburb]
        ,[Call_Postcode]
        ,[Call_Date]
        ,[Call_Details]
        ,[Call_Type]
        ,[Call_Objectives]
        ,[Off_Road_Days]
        ,[Off_Road_Reason]
        ,[Call_Status]
        ,[Contact_Name]
        ,[Customer_Type]
        ,[CreatedBy_ID]
        ,[Created_Date]
        ,[Last_ModifiedBy_ID]
        ,[Last_Modified_Date]
        )

        VALUES

        ({ID}
        ,{Account_ID}
        ,{Site_SalesRep_ID}
        ,{Call_Owner_ID}
        ,{Site_ID}
        ,{Contact_ID}
        ,{Call_Name}
        ,{Call_Street}
        ,{Call_Suburb}
        ,{Call_Postcode}
        ,{Call_Date}
        ,{Call_Details}
        ,{Call_Type}
        ,{Call_Objectives}
        ,{Off_Road_Days}
        ,{Off_Road_Reason}
        ,{Call_Status}
        ,{Contact_Name}
        ,{Customer_Type}
        ,{CreatedBy_ID}
        ,{Created_Date}
        ,{Last_ModifiedBy_ID}
        ,{Last_Modified_Date}
        ); 

        """

        call_inserts += insert
        

    # Execute each SQL query in the list
        cursor.execute(insert)

        conn.commit()


    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
print('The new lines were inserted into SQL!')

#### Updating existing lines

In [23]:
if df_calls_updated.empty:
    calls_finish_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    pass
        
else:

    # Establish the connection with Windows authentication
    conn = pyodbc.connect(
        f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_sql};DATABASE={database_sql};UID={username_sql};PWD={password_sql};'
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()
    

    df_up_callLines_SQL = df_up_callLines_SQL[~mask_calls]

    df_up_callLines_SQL = df_up_callLines_SQL.sort_values(by='ID').reset_index(drop=True)

    df_calls_updated = df_calls_updated.replace(pd.NA, None).sort_values(by='Id').reset_index(drop=True)

    df_call_compare = df_calls_updated.rename(columns=callLines_column_names)
    
    
    for index, item in df_call_compare.iterrows():
        
        differences = df_call_compare.loc[index].fillna('NULL').astype(str) != df_up_callLines_SQL.iloc[index].fillna('NULL').astype(str)
        
        columns_with_difference = differences[differences].index.tolist()
        
        if columns_with_difference:
            
        # Construct the update statement for SQL Server
            update_statement = "UPDATE [Sample_Salesforce].[dbo].[SF_F2FCalls] SET "
        
            for column in columns_with_difference:
                value = df_call_compare.loc[index, column]
                if isinstance(value, np.bool_):
                    value = 1 if value else 0
                elif value is None:
                    value = 'NULL'
                else:
                    value = value.replace("'", "''")
                    value = f"'{value}'"
                    
                update_statement += f"{column} = {value}, "
           
        
            update_statement = update_statement[:-2]  # Remove the trailing comma and space
            update_statement += f" WHERE ID = '{df_call_compare.loc[index, 'ID']}';"  # Assuming there's an 'ID' column in your DataFrame
            
            # Execute each SQL query in the list
            cursor.execute(update_statement)

            conn.commit()


    # Close the cursor and connection
    cursor.close()
    conn.close()
    calls_finish_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [24]:
print('The lines with changes were updated in SQL!')

The lines with changes were updated in SQL!


### Updating the Jobs table

In [25]:
TableName = 'F2FCalls'

Last_Create_Date = last_created_call_date_extra if df_calls_created.empty else df_calls_created['CreatedDate'].max()
Last_Create_Date = pd.to_datetime(Last_Create_Date)
Last_Create_Date = Last_Create_Date.strftime('%Y-%m-%d %H:%M:%S')

Last_Modified_Date = last_modified_call_date_extra if df_calls_updated.empty else df_calls_updated['LastModifiedDate'].max()
Last_Modified_Date = pd.to_datetime(Last_Modified_Date)
Last_Modified_Date.strftime('%Y-%m-%d %H:%M:%S')

total_time = datetime.strptime(calls_finish_time, '%Y-%m-%d %H:%M:%S') - datetime.strptime(calls_start_time, '%Y-%m-%d %H:%M:%S')

no_rows = df_calls_created.shape[0] + df_calls_updated.shape[0]

In [26]:
# Establish the connection with Windows authentication
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_sql};DATABASE={database_sql};UID={username_sql};PWD={password_sql};'
)

# Create a cursor object to interact with the database
cursor = conn.cursor()

insert = f"""
INSERT INTO [Sample_Salesforce].[dbo].[SF_InsertJobs_Log]
           ([Table_Name]
           ,[LastLineCreated_Date]
           ,[LastLineModified_Date]
           ,[JobStart_Date]
           ,[JobFinish_Date]
           ,[TotalTime]
           ,[No_Rows])
     VALUES
           ('{TableName}'
           ,'{Last_Create_Date}'
           ,'{Last_Modified_Date}'
           ,'{calls_start_time}'
           ,'{calls_finish_time}'
           ,'{total_time}'
           ,'{no_rows}')
"""

# Execute each SQL query in the list
cursor.execute(insert)

conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [27]:
print('The script is over. All lines inserted and updated!')

The script is over. All lines inserted and updated!


In [28]:
time.sleep(5)